In [0]:
# This library allows the Python kernel to stream content to an Event Hub:
%pip install azure-eventhub

Python interpreter will be restarted.
Requirement already satisfied: azure-eventhub in /local_disk0/.ephemeral_nfs/envs/pythonEnv-b256d946-69ee-4600-950a-30afc12e0653/lib/python3.8/site-packages (5.10.1)
Requirement already satisfied: typing-extensions>=4.0.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-b256d946-69ee-4600-950a-30afc12e0653/lib/python3.8/site-packages (from azure-eventhub) (4.4.0)
Requirement already satisfied: azure-core<2.0.0,>=1.14.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-b256d946-69ee-4600-950a-30afc12e0653/lib/python3.8/site-packages (from azure-eventhub) (1.26.1)
Requirement already satisfied: uamqp<2.0.0,>=1.6.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-b256d946-69ee-4600-950a-30afc12e0653/lib/python3.8/site-packages (from azure-eventhub) (1.6.3)
Requirement already satisfied: requests>=2.18.4 in /databricks/python3/lib/python3.8/site-packages (from azure-core<2.0.0,>=1.14.0->azure-eventhub) (2.25.1)
Requirement already satisfied: six>=1.11.0 in /databricks/python3/lib/python3.8/site-packages (from azure-core<2.0.0,>=1.14.0->azure-eventhub) (1.15.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.14.0->azure-eventhub) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.14.0->azure-eventhub) (2.10)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.14.0->azure-eventhub) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.14.0->azure-eventhub) (4.0.0)
WARNING: You are using pip version 21.0.1; however, version 22.3.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-b256d946-69ee-4600-950a-30afc12e0653/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
event_hub_connection_string = "Endpoint=sb://streamingeventhub2.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=rzs1q+a2bgXfiCgxv7cpiCoUZv4dO2stncZOGOVHq7o=" # Paste your Event Hubs connection string in the quotes to the left
event_hub_name = "airlinehub"
connection_string = event_hub_connection_string + ";EntityPath=" + event_hub_name

print("Consumer Connection String: {}".format(connection_string))

Consumer Connection String: Endpoint=sb://streamingeventhub2.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=rzs1q+a2bgXfiCgxv7cpiCoUZv4dO2stncZOGOVHq7o=;EntityPath=airlinehub

In [0]:
%python

import json

# Create the starting position Dictionary
startingEventPosition = {
  "offset": "-1",
  "seqNo": -1,            # not in use
  "enqueuedTime": None,   # not in use
  "isInclusive": True
}

eventHubsConf = {
  "eventhubs.connectionString" : sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connection_string),
  "eventhubs.startingPosition" : json.dumps(startingEventPosition),
  "setMaxEventsPerTrigger": 10
}

In [0]:
%python

from pyspark.sql.functions import col

spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)

eventStreamDF = (spark.readStream
  .format("eventhubs")
  .options(**eventHubsConf)
  .load()
)

eventStreamDF.printSchema()

root
-- body: binary (nullable = true)
-- partition: string (nullable = true)
-- offset: string (nullable = true)
-- sequenceNumber: long (nullable = true)
-- enqueuedTime: timestamp (nullable = true)
-- publisher: string (nullable = true)
-- partitionKey: string (nullable = true)
-- properties: map (nullable = true)
 |-- key: string
 |-- value: string (valueContainsNull = true)
-- systemProperties: map (nullable = true)
 |-- key: string
 |-- value: string (valueContainsNull = true)

In [0]:
%python
bodyDF = eventStreamDF.select(col("body").cast("STRING"))

In [0]:
%python
display(bodyDF, streamName= "bodyDF")

body
"{""MONTH"": ""2"", ""DAY_OF_MONTH"": ""17"", ""DAY_OF_WEEK"": ""7"", ""OP_UNIQUE_CARRIER"": ""AA"", ""TAIL_NUM"": ""N896NN"", ""OP_CARRIER_FL_NUM"": ""110"", ""ORIGIN_AIRPORT_ID"": ""11057"", ""ORIGIN"": ""CLT"", ""ORIGIN_CITY_NAME"": ""Charlotte, NC"", ""DEST_AIRPORT_ID"": ""10693"", ""DEST"": ""BNA"", ""DEST_CITY_NAME"": ""Nashville, TN"", ""CRS_DEP_TIME"": ""2246"", ""DEP_TIME"": ""2247"", ""DEP_DELAY_NEW"": ""1.00"", ""DEP_DEL15"": ""0.00"", ""DEP_TIME_BLK"": ""2200-2259"", ""CRS_ARR_TIME"": ""2313"", ""ARR_TIME"": ""2332"", ""ARR_DELAY_NEW"": ""19.00"", ""ARR_TIME_BLK"": ""2300-2359"", ""CANCELLED"": ""0.00"", ""CANCELLATION_CODE"": """", ""CRS_ELAPSED_TIME"": ""87.00"", ""ACTUAL_ELAPSED_TIME"": ""105.00"", ""DISTANCE"": ""328.00"", ""DISTANCE_GROUP"": ""2"", ""CARRIER_DELAY"": ""1.00"", ""WEATHER_DELAY"": ""0.00"", ""NAS_DELAY"": ""18.00"", ""SECURITY_DELAY"": ""0.00"", ""LATE_AIRCRAFT_DELAY"": ""0.00""}"
"{""MONTH"": ""2"", ""DAY_OF_MONTH"": ""19"", ""DAY_OF_WEEK"": ""2"", ""OP_UNIQUE_CARRIER"": ""AA"", ""TAIL_NUM"": ""N905AN"", ""OP_CARRIER_FL_NUM"": ""110"", ""ORIGIN_AIRPORT_ID"": ""11057"", ""ORIGIN"": ""CLT"", ""ORIGIN_CITY_NAME"": ""Charlotte, NC"", ""DEST_AIRPORT_ID"": ""10693"", ""DEST"": ""BNA"", ""DEST_CITY_NAME"": ""Nashville, TN"", ""CRS_DEP_TIME"": ""2246"", ""DEP_TIME"": ""2303"", ""DEP_DELAY_NEW"": ""17.00"", ""DEP_DEL15"": ""1.00"", ""DEP_TIME_BLK"": ""2200-2259"", ""CRS_ARR_TIME"": ""2313"", ""ARR_TIME"": ""2349"", ""ARR_DELAY_NEW"": ""36.00"", ""ARR_TIME_BLK"": ""2300-2359"", ""CANCELLED"": ""0.00"", ""CANCELLATION_CODE"": """", ""CRS_ELAPSED_TIME"": ""87.00"", ""ACTUAL_ELAPSED_TIME"": ""106.00"", ""DISTANCE"": ""328.00"", ""DISTANCE_GROUP"": ""2"", ""CARRIER_DELAY"": ""17.00"", ""WEATHER_DELAY"": ""0.00"", ""NAS_DELAY"": ""19.00"", ""SECURITY_DELAY"": ""0.00"", ""LATE_AIRCRAFT_DELAY"": ""0.00""}"
"{""MONTH"": ""2"", ""DAY_OF_MONTH"": ""21"", ""DAY_OF_WEEK"": ""4"", ""OP_UNIQUE_CARRIER"": ""AA"", ""TAIL_NUM"": ""N913AN"", ""OP_CARRIER_FL_NUM"": ""110"", ""ORIGIN_AIRPORT_ID"": ""11057"", ""ORIGIN"": ""CLT"", ""ORIGIN_CITY_NAME"": ""Charlotte, NC"", ""DEST_AIRPORT_ID"": ""10693"", ""DEST"": ""BNA"", ""DEST_CITY_NAME"": ""Nashville, TN"", ""CRS_DEP_TIME"": ""2246"", ""DEP_TIME"": ""2329"", ""DEP_DELAY_NEW"": ""43.00"", ""DEP_DEL15"": ""1.00"", ""DEP_TIME_BLK"": ""2200-2259"", ""CRS_ARR_TIME"": ""2313"", ""ARR_TIME"": ""2351"", ""ARR_DELAY_NEW"": ""38.00"", ""ARR_TIME_BLK"": ""2300-2359"", ""CANCELLED"": ""0.00"", ""CANCELLATION_CODE"": """", ""CRS_ELAPSED_TIME"": ""87.00"", ""ACTUAL_ELAPSED_TIME"": ""82.00"", ""DISTANCE"": ""328.00"", ""DISTANCE_GROUP"": ""2"", ""CARRIER_DELAY"": ""3.00"", ""WEATHER_DELAY"": ""0.00"", ""NAS_DELAY"": ""0.00"", ""SECURITY_DELAY"": ""0.00"", ""LATE_AIRCRAFT_DELAY"": ""35.00""}"
"{""MONTH"": ""2"", ""DAY_OF_MONTH"": ""23"", ""DAY_OF_WEEK"": ""6"", ""OP_UNIQUE_CARRIER"": ""AA"", ""TAIL_NUM"": ""N946NN"", ""OP_CARRIER_FL_NUM"": ""110"", ""ORIGIN_AIRPORT_ID"": ""11057"", ""ORIGIN"": ""CLT"", ""ORIGIN_CITY_NAME"": ""Charlotte, NC"", ""DEST_AIRPORT_ID"": ""10693"", ""DEST"": ""BNA"", ""DEST_CITY_NAME"": ""Nashville, TN"", ""CRS_DEP_TIME"": ""2246"", ""DEP_TIME"": ""2240"", ""DEP_DELAY_NEW"": ""0.00"", ""DEP_DEL15"": ""0.00"", ""DEP_TIME_BLK"": ""2200-2259"", ""CRS_ARR_TIME"": ""2313"", ""ARR_TIME"": ""2307"", ""ARR_DELAY_NEW"": ""0.00"", ""ARR_TIME_BLK"": ""2300-2359"", ""CANCELLED"": ""0.00"", ""CANCELLATION_CODE"": """", ""CRS_ELAPSED_TIME"": ""87.00"", ""ACTUAL_ELAPSED_TIME"": ""87.00"", ""DISTANCE"": ""328.00"", ""DISTANCE_GROUP"": ""2"", ""CARRIER_DELAY"": """", ""WEATHER_DELAY"": """", ""NAS_DELAY"": """", ""SECURITY_DELAY"": """", ""LATE_AIRCRAFT_DELAY"": """"}"
"{""MONTH"": ""2"", ""DAY_OF_MONTH"": ""25"", ""DAY_OF_WEEK"": ""1"", ""OP_UNIQUE_CARRIER"": ""AA"", ""TAIL_NUM"": ""N907NN"", ""OP_CARRIER_FL_NUM"": ""110"", ""ORIGIN_AIRPORT_ID"": ""11057"", ""ORIGIN"": ""CLT"", ""ORIGIN_CITY_NAME"": ""Charlotte, NC"", ""DEST_AIRPORT_ID"": ""10693"", ""DEST"": ""BNA"", ""DEST

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, DoubleType

schema = StructType([
      StructField("MONTH", StringType(), True),
      StructField("DAY_OF_MONTH", StringType(), True),
      StructField("DAY_OF_WEEK", StringType(), True),
      StructField("OP_UNIQUE_CARRIER", StringType(), True),
      StructField("TAIL_NUM", StringType(), True),
      StructField("OP_CARRIER_FL_NUM", StringType(), True),
      StructField("ORIGIN_AIRPORT_ID", StringType(), True),
      StructField("ORIGIN", StringType(), True),
      StructField("ORIGIN_CITY_NAME", StringType(), True),
      StructField("DEST_AIRPORT_ID", StringType(), True),
      StructField("DEST", StringType(), True),
      StructField("DEST_CITY_NAME", StringType(), True),
      StructField("CRS_DEP_TIME", StringType(), True), 
      StructField("DEP_TIME", StringType(), True), 
      StructField("DEP_DELAY_NEW", StringType(), True), 
      StructField("DEP_DEL15", StringType(), True) ,
      StructField("DEP_TIME_BLK", StringType(), True), 
      StructField("CRS_ARR_TIME", StringType(), True) ,
      StructField("ARR_TIME", StringType(), True) ,
      StructField("ARR_DELAY_NEW", StringType(), True), 
      StructField("ARR_TIME_BLK", StringType(), True) ,
      StructField("CANCELLED", StringType(), True) ,
      StructField("CANCELLATION_CODE", StringType(), True) ,
      StructField("CRS_ELAPSED_TIME", StringType(), True) ,
      StructField("ACTUAL_ELAPSED_TIME", StringType(), True) ,
      StructField("DISTANCE", StringType(), True) ,
      StructField("DISTANCE_GROUP", StringType(), True) ,
      StructField("CARRIER_DELAY", StringType(), True) ,
      StructField("WEATHER_DELAY", StringType(), True) ,
      StructField("NAS_DELAY", StringType(), True) ,
      StructField("SECURITY_DELAY", StringType(), True) ,
      StructField("LATE_AIRCRAFT_DELAY", StringType(), True)
])

In [0]:
%python

from pyspark.sql.functions import col, from_json

parsedEventsDF = bodyDF.select(
  from_json(col("body"), schema).alias("json"))

parsedEventsDF.printSchema()

root
-- json: struct (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY_OF_MONTH: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- OP_UNIQUE_CARRIER: string (nullable = true)
 |-- TAIL_NUM: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: string (nullable = true)
 |-- ORIGIN_AIRPORT_ID: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY_NAME: string (nullable = true)
 |-- DEST_AIRPORT_ID: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY_NAME: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEP_DELAY_NEW: string (nullable = true)
 |-- DEP_DEL15: string (nullable = true)
 |-- DEP_TIME_BLK: string (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- ARR_TIME: string (nullable = true)
 |-- ARR_DELAY_NEW: string (nullable = true)
 |-- ARR_TIME_BLK: string (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- CRS_ELAPSED_TIME: string (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- DISTANCE_GROUP: string (nullable = true)
 |-- CARRIER_DELAY: string (nullable = true)
 |-- WEATHER_DELAY: string (nullable = true)
 |-- NAS_DELAY: string (nullable = true)
 |-- SECURITY_DELAY: string (nullable = true)
 |-- LATE_AIRCRAFT_DELAY: string (nullable = true)

In [0]:
%python

flatSchemaDF = (parsedEventsDF
  .select(col("json.MONTH").alias("Month"),
          col("json.DAY_OF_MONTH").alias("Day_Of_Month"),
          col("json.DAY_OF_WEEK").alias("Day_Of_Week"),
          col("json.OP_UNIQUE_CARRIER").alias("OP_Unique_Carrier"),
          col("json.TAIL_NUM").alias("Tail_Num"),
          col("json.OP_CARRIER_FL_NUM").alias("OP_CARRIER_FL_NUM"),
          col("json.ORIGIN_AIRPORT_ID").alias("ORIGIN_AIRPORT_ID"),
          col("json.ORIGIN").alias("ORIGIN"),
          col("json.ORIGIN_CITY_NAME").alias("ORIGIN_CITY_NAME"),
          col("json.DEST_AIRPORT_ID").alias("DEST_AIRPORT_ID"),
          col("json.DEST").alias("DEST"),
          col("json.DEST_CITY_NAME").alias("DEST_CITY_NAME"),
          col("json.CRS_DEP_TIME").alias("CRS_DEP_TIME"),
          col("json.DEP_TIME").alias("DEP_TIME"),
          col("json.DEP_DELAY_NEW").alias("DEP_DELAY_NEW"),
          col("json.DEP_DEL15").alias("DEP_DEL15"),
          col("json.DEP_TIME_BLK").alias("DEP_TIME_BLK"),
          col("json.CRS_ARR_TIME").alias("CRS_ARR_TIME"),
          col("json.ARR_TIME").alias("ARR_TIME"),
          col("json.ARR_DELAY_NEW").alias("ARR_DELAY_NEW"),
          col("json.ARR_TIME_BLK").alias("ARR_TIME_BLK"),
          col("json.CANCELLED").alias("CANCELLED"),
          col("json.CANCELLATION_CODE").alias("CANCELLATION_CODE"),
          col("json.CRS_ELAPSED_TIME").alias("CRS_ELAPSED_TIME"),
          col("json.ACTUAL_ELAPSED_TIME").alias("ACTUAL_ELAPSED_TIME"),
          col("json.DISTANCE").alias("DISTANCE"),
          col("json.DISTANCE_GROUP").alias("DISTANCE_GROUP"),
          col("json.CARRIER_DELAY").alias("CARRIER_DELAY"),
          col("json.WEATHER_DELAY").alias("WEATHER_DELAY"),
          col("json.NAS_DELAY").alias("NAS_DELAY"),
          col("json.SECURITY_DELAY").alias("SECURITY_DELAY"),
          col("json.LATE_AIRCRAFT_DELAY").alias("LATE_AIRCRAFT_DELAY") )
)

In [0]:
%python
display(flatSchemaDF)

Month,Day_Of_Month,Day_Of_Week,OP_Unique_Carrier,Tail_Num,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,CRS_ARR_TIME,ARR_TIME,ARR_DELAY_NEW,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
2,17,7,AA,N896NN,110,11057,CLT,"Charlotte, NC",10693,BNA,"Nashville, TN",2246,2247,1.00,0.00,2200-2259,2313,2332,19.00,2300-2359,0.00,,87.00,105.00,328.00,2,1.00,0.00,18.00,0.00,0.00
2,19,2,AA,N905AN,110,11057,CLT,"Charlotte, NC",10693,BNA,"Nashville, TN",2246,2303,17.00,1.00,2200-2259,2313,2349,36.00,2300-2359,0.00,,87.00,106.00,328.00,2,17.00,0.00,19.00,0.00,0.00
2,21,4,AA,N913AN,110,11057,CLT,"Charlotte, NC",10693,BNA,"Nashville, TN",2246,2329,43.00,1.00,2200-2259,2313,2351,38.00,2300-2359,0.00,,87.00,82.00,328.00,2,3.00,0.00,0.00,0.00,35.00
2,23,6,AA,N946NN,110,11057,CLT,"Charlotte, NC",10693,BNA,"Nashville, TN",2246,2240,0.00,0.00,2200-2259,2313,2307,0.00,2300-2359,0.00,,87.00,87.00,328.00,2,,,,,
2,25,1,AA,N907NN,110,11057,CLT,"Charlotte, NC",10693,BNA,"Nashville, TN",2246,2239,0.00,0.00,2200-2259,2313,2310,0.00,2300-2359,0.00,,87.00,91.00,328.00,2,,,,,
2,27,3,AA,N917NN,110,11057,CLT,"Charlotte, NC",10693,BNA,"Nashville, TN",2246,2241,0.00,0.00,2200-2259,2313,2310,0.00,2300-2359,0.00,,87.00,89.00,328.00,2,,,,,
2,5,2,AA,N912AN,110,11057,CLT,"Charlotte, NC",12339,IND,"Indianapolis, IN",1654,1648,0.00,0.00,1600-1659,1836,1812,0.00,1800-1859,0.00,,102.00,84.00,427.00,2,,,,,
2,1,5,AA,N907AN,111,12339,IND,"Indianapolis, IN",11057,CLT,"Charlotte, NC",0622,0620,0.00,0.00,0600-0659,0814,0914,60.00,0800-0859,0.00,,112.00,174.00,427.00,2,0.00,0.00,60.00,0.00,0.00
2,5,2,AA,N862NN,111,12339,IND,"Indianapolis, IN",11057,CLT,"Charlotte, NC",0622,0616,0.00,0.00,0600-0659,0814,0759,0.00,0800-0859,0.00,,112.00,103.00,427.00,2,,,,,
2,7,4,AA,N909AN,111,12339,IND,"Indianapolis, IN",11057,CLT,"Charlotte, NC",0622,0623,1.00,0.00,0600-0659,0814,0817,3.00,0800-0859,0.00,,112.00,114.00,427.00,2,,,,,


In [0]:
flatSchemaDF.createOrReplaceTempView("airlineEvents55")

In [0]:
%sql
Select * from airlineEvents55 limit 10

Month,Day_Of_Month,Day_Of_Week,OP_Unique_Carrier,Tail_Num,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,CRS_ARR_TIME,ARR_TIME,ARR_DELAY_NEW,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
2,17,7,AA,N896NN,110,11057,CLT,"Charlotte, NC",10693,BNA,"Nashville, TN",2246,2247,1.00,0.00,2200-2259,2313,2332,19.00,2300-2359,0.00,,87.00,105.00,328.00,2,1.00,0.00,18.00,0.00,0.00
2,19,2,AA,N905AN,110,11057,CLT,"Charlotte, NC",10693,BNA,"Nashville, TN",2246,2303,17.00,1.00,2200-2259,2313,2349,36.00,2300-2359,0.00,,87.00,106.00,328.00,2,17.00,0.00,19.00,0.00,0.00
2,21,4,AA,N913AN,110,11057,CLT,"Charlotte, NC",10693,BNA,"Nashville, TN",2246,2329,43.00,1.00,2200-2259,2313,2351,38.00,2300-2359,0.00,,87.00,82.00,328.00,2,3.00,0.00,0.00,0.00,35.00
2,23,6,AA,N946NN,110,11057,CLT,"Charlotte, NC",10693,BNA,"Nashville, TN",2246,2240,0.00,0.00,2200-2259,2313,2307,0.00,2300-2359,0.00,,87.00,87.00,328.00,2,,,,,
2,25,1,AA,N907NN,110,11057,CLT,"Charlotte, NC",10693,BNA,"Nashville, TN",2246,2239,0.00,0.00,2200-2259,2313,2310,0.00,2300-2359,0.00,,87.00,91.00,328.00,2,,,,,
2,27,3,AA,N917NN,110,11057,CLT,"Charlotte, NC",10693,BNA,"Nashville, TN",2246,2241,0.00,0.00,2200-2259,2313,2310,0.00,2300-2359,0.00,,87.00,89.00,328.00,2,,,,,
2,5,2,AA,N912AN,110,11057,CLT,"Charlotte, NC",12339,IND,"Indianapolis, IN",1654,1648,0.00,0.00,1600-1659,1836,1812,0.00,1800-1859,0.00,,102.00,84.00,427.00,2,,,,,
2,1,5,AA,N907AN,111,12339,IND,"Indianapolis, IN",11057,CLT,"Charlotte, NC",0622,0620,0.00,0.00,0600-0659,0814,0914,60.00,0800-0859,0.00,,112.00,174.00,427.00,2,0.00,0.00,60.00,0.00,0.00
2,5,2,AA,N862NN,111,12339,IND,"Indianapolis, IN",11057,CLT,"Charlotte, NC",0622,0616,0.00,0.00,0600-0659,0814,0759,0.00,0800-0859,0.00,,112.00,103.00,427.00,2,,,,,
2,7,4,AA,N909AN,111,12339,IND,"Indianapolis, IN",11057,CLT,"Charlotte, NC",0622,0623,1.00,0.00,0600-0659,0814,0817,3.00,0800-0859,0.00,,112.00,114.00,427.00,2,,,,,


In [0]:
%sql
Select count(1) from airlineEvents55

count(1)
3025


In [0]:
final_df = flatSchemaDF.select("DAY_OF_MONTH","DAY_OF_WEEK","OP_UNIQUE_CARRIER","TAIL_NUM","ORIGIN","DEST","DEST_CITY_NAME","DEP_TIME","ARR_TIME","CANCELLED")

In [0]:
final_df \
 .writeStream \
 .format("csv") \
 .option("path", "/FileStore/tables/airlinedata") \
 .option("header", "true") \
 .option("checkpointLocation", "/tmp/checkpoint_path") \
 .outputMode("append") \
 .start()

Out[14]: <pyspark.sql.streaming.StreamingQuery at 0x7f0cecd43220>

In [0]:
%sql
DROP TABLE IF EXISTS airlineFlightsData;

/* Testing one of the file that was created as Batch into the location /FileStore/tables/airlinesink */

CREATE TABLE airlineFlightsData
USING csv
OPTIONS (path "/FileStore/tables/airlinedata/part-00000-6b6086ad-5353-402e-ac4f-6deba160ea22-c000.csv", header "true")


In [0]:
%sql
Select * from airlineFlightsData

DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,DEST,DEST_CITY_NAME,DEP_TIME,ARR_TIME,CANCELLED
14,4,AA,N339PL,DFW,MCI,"Kansas City, MO",0935,1112,0.00
17,7,AA,N93003,DFW,MCI,"Kansas City, MO",0954,1108,0.00
19,2,AA,N861NN,DFW,MCI,"Kansas City, MO",0936,1049,0.00
21,4,AA,N971NN,DFW,MCI,"Kansas City, MO",0944,1119,0.00
24,7,AA,N9006,DFW,MCI,"Kansas City, MO",0932,1056,0.00
26,2,AA,N997NN,DFW,MCI,"Kansas City, MO",0936,1058,0.00
28,4,AA,N951NN,DFW,MCI,"Kansas City, MO",0941,1128,0.00
15,5,AA,N933NN,MCI,DFW,"Dallas/Fort Worth, TX",1230,1453,0.00
18,1,AA,N952AA,MCI,DFW,"Dallas/Fort Worth, TX",1150,1335,0.00
20,3,AA,N829NN,MCI,DFW,"Dallas/Fort Worth, TX",1153,1339,0.00
